In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import os
import aurora_data_api
from dateutil import parser

In [39]:
nameMap = {
    'MEGAN': "Megan Schacht-bonneau",
    'Megan': "Megan Schacht-bonneau",
    "JORDYS": "Jordys Placido",
    "Jordys": "Jordys Placido",
    "NIKKY": "Nikky Voeuth",
    "Nikky": "Nikky Voeuth"
}
resultsMap = {
    'PASSED': 'Pass',
    'FAILED': 'Fail',
    'PASS': 'Pass',
    'FAIL': 'Fail'
}

def getDate(inp):
#     print('inp', inp)
    dateVal = re.findall('[0-9][0-9]?/..?/2?0?21', inp)[0]
    formatted = parser.parse(dateVal).strftime('%Y-%m-%d')
    return formatted

# Assume that 
def getFailureType(inp):
    if inp[0] == 'W':
        return inp[:6]
    else:
        return 'none'
    
def getBatchNumber(inp):
    mostRecentBatch = inp.iloc[0]
    for i in range(len(inp)):
        val = inp.iloc[i]
        if type(val) == str:
            mostRecentBatch = val.split(' ')[1]
        inp.iloc[i] = mostRecentBatch
    return inp

def getResults(inp):
    return resultsMap.get(inp, inp)

def getTester(inp):
    return nameMap.get(inp, inp)

# Execute a SQL query: helper method to be used by insertEntry
def execute(q):
    try:
        db = aurora_data_api.connect(aurora_cluster_arn = r_ARN, secret_arn=s_ARN, database=dbName)
        c = db.cursor()
        c.execute(q) # Execute the query
        db.commit()
        return c.fetchall() # Display the output of said query
    except Exception as e: # Error occurred during execute
        print('Error with execute:', e)
    
        return (float('inf'), e)
    
def insertEntry(sn, bn, date, tst, ft, fail, msg, dt, fw):
    try:
        q = ''' Insert into QAResults values ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')''' % (sn, bn, date, tst, ft, fail, msg, dt, fw)
        return execute(q)
    except Exception as e:
        print('serialNum', sn)
        print('Issue with insert entry:', e)
    
# Read the path in--get a table formatted to more easily insert entries
def readPath(path, deviceType):
    df = pd.read_excel(path)
    
    if 'Unnamed: 1' in df.columns:
        df = df.dropna(subset = ['Unnamed: 1'])
        df = df[df['Unnamed: 0'] != 'BATCH']
        if 'Unnamed: 5' in df.columns:
            df = df.drop(columns = ['Unnamed: 5'])
    df.columns = ['BatchNumber', 'SerialNumber', 'FunctionalTest', 'LogMessage', 'Tester']
#     display(df.head())
    df['BatchNumber'] = getBatchNumber(df['BatchNumber'])
    df = df[df['FunctionalTest'] != 'RESULT']
    df['FunctionalTest'] = df['FunctionalTest'].apply(getResults)
    df['Tester'] = df['Tester'].apply(getTester)
#     display(df)
    df['Date'] = df['LogMessage'].apply(getDate)
    df['FailureType'] = df['LogMessage'].apply(getFailureType)
    df['DeviceType'] = deviceType
    
    return df

paths = []
directories = [d for d in os.listdir('.') if d[-1] == 'x' and '$' not in d]
display(directories)

['MINI QC Test 01.10 01.16 2021.xlsx',
 'Mini QC Test 07.04 to 07.10 2021.xlsx',
 'MINI QC Test 01.17 01.23 2021.xlsx',
 'MINI QC Test 01.24 01.30 2021.xlsx',
 'Mini QC Test 07.11 to 07.17 2021.xlsx',
 'MINI QC Test 03.21 03.27 2021.xlsx',
 'MINI QC Test 06.06 06.12 2021.xlsx',
 'MINI QC Test 05.30 06.05 2021.xlsx',
 'MINI QC Test 04.04 04.10 2021.xlsx',
 'MINI QC Test 05.16 05.22 2021.xlsx',
 'MINI QC Test 05.23 05.29 2021.xlsx',
 'MINI QC Test 03.14 03.20 2021.xlsx',
 'MINI QC Test 02.07 02.13 2021.xlsx',
 'MINI QC Test 04.25 05.01 2021.xlsx',
 'MINI QC Test 01.31 02.06 2021.xlsx',
 'Mini QC Test 06.13 to 06.19 2021.xlsx',
 'MINI QC Test 03.28 04.03 2021.xlsx',
 'MINI QC Test 04.11 04.17 2021.xlsx',
 'MINI QC Test 02.28 03.06 2021.xlsx',
 'MINI QC Test 03.07 03.13 2021.xlsx',
 'MINI QC Test 02.14 02.20 2021.xlsx',
 'MINI QC Test 06.20 06.26 2021.xlsx',
 'Mini QC Test 07.18 to 07.24 2021.xlsx',
 'Mini QC Test 06.27 to 07.03 2021.xlsx',
 'MINI QC Test 01.03 01.09 2021.xlsx',
 'Mini QC 

In [46]:
port = 3306
user = 'admin'
password = 'password'

host = 'dashboarddb.cluster-cyvmmsfjcmrl.us-east-1.rds.amazonaws.com'
dbName = 'db'
r_ARN = 'arn:aws:rds:us-east-1:319481862133:cluster:dashboarddb'
s_ARN = 'arn:aws:secretsmanager:us-east-1:319481862133:secret:dashboardDBKey-6XBatB'
def insertTable(res):
    for i in range(len(res)):
        row = res.iloc[i]
        bn = row['BatchNumber']
        sn = row['SerialNumber']
        tst = row['Tester']
        msg = row['LogMessage']
        ft = row['FunctionalTest']
        fail = row['FailureType']
        date = row['Date']
        dt = row['DeviceType']
        fw = '9.70'
        insertEntry(sn, bn, date, tst, ft, fail, msg, dt, fw)
    print('success!')

In [43]:
# July Files
julyFiles = [d for d in directories if d[14] == '7']
julyTables = []
for path in julyFiles:
    julyTables.append(readPath(path, 'Mini'))
for p in julyTables:
    display(p.head())
for t in julyTables:
    insertTable(t)

,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA2021-05-05,G8V190001173228P,Pass,COMPLETED (FW 9.70) RESET 7/6/21,Jordys Placido,2021-07-06,none,Mini
2,NA2021-05-05,G8V190001173227V,Pass,COMPLETED (FW 9.70) RESET 7/6/21,Jordys Placido,2021-07-06,none,Mini
3,NA2021-05-05,G8V19000117322A7,Pass,COMPLETED (FW 9.70) RESET 7/6/21,Jordys Placido,2021-07-06,none,Mini
4,NA2021-05-05,G8V190001173225R,Pass,COMPLETED (FW 9.70) RESET 7/6/21,Jordys Placido,2021-07-06,none,Mini
5,NA2021-05-05,G8V190001173229F,Pass,COMPLETED (FW 9.70) RESET 7/6/21,Jordys Placido,2021-07-06,none,Mini


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA2021-06-28,G8V190001173CB09,Pass,COMPLETED (FW 9.70) RESET 7/12/21,Jordys Placido,2021-07-12,none,Mini
2,NA2021-06-28,G8V190001173CBFS,Pass,COMPLETED (FW 9.70) RESET 7/12/21,Jordys Placido,2021-07-12,none,Mini
3,NA2021-06-28,G8V190001173CBG4,Pass,COMPLETED (FW 9.70) RESET 7/12/21,Jordys Placido,2021-07-12,none,Mini
4,NA2021-06-28,G8V190001173CB0T,Pass,COMPLETED (FW 9.70) RESET 7/12/21,Jordys Placido,2021-07-12,none,Mini
5,NA2021-06-28,G8V190001173CBDX,Pass,COMPLETED (FW 9.70) RESET 7/12/21,Jordys Placido,2021-07-12,none,Mini


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA2021-04-19,G8V19000113278VF,Pass,COMPLETED (FW 9.70) RESET 7/19/21,Jordys Placido,2021-07-19,none,Mini
2,NA2021-04-19,G8V190001132740G,Pass,COMPLETED (FW 9.70) RESET 7/19/21,Jordys Placido,2021-07-19,none,Mini
3,NA2021-04-19,G8V19000113273UR,Pass,COMPLETED (FW 9.70) RESET 7/19/21,Jordys Placido,2021-07-19,none,Mini
4,NA2021-04-19,G8V190001132793M,Pass,COMPLETED (FW 9.70) RESET 7/19/21,Jordys Placido,2021-07-19,none,Mini
5,NA2021-04-19,G8V1900011327961,Pass,COMPLETED (FW 9.70) RESET 7/19/21,Jordys Placido,2021-07-19,none,Mini


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA2021-07-05,G8V190001173D4W2,Pass,COMPLETED (FW 9.70) RESET 7/27/21,Jordys Placido,2021-07-27,none,Mini
2,NA2021-07-05,G8V190001173D4X5,Pass,COMPLETED (FW 9.70) RESET 7/27/21,Jordys Placido,2021-07-27,none,Mini
3,NA2021-07-05,G8V190001173D4WA,Pass,COMPLETED (FW 9.70) RESET 7/27/21,Jordys Placido,2021-07-27,none,Mini
4,NA2021-07-05,G8V190001173D4WW,Pass,COMPLETED (FW 9.70) RESET 7/27/21,Jordys Placido,2021-07-27,none,Mini
5,NA2021-07-05,G8V190001173D4X0,Pass,COMPLETED (FW 9.70) RESET 7/27/21,Jordys Placido,2021-07-27,none,Mini


success!
success!
success!
success!


In [47]:
# June Files
juneFiles = [d for d in directories if d[14] == '6']
juneTables = []
for path in juneFiles:
    juneTables.append(readPath(path, 'Mini'))
for p in juneTables:
    display(p.head())
for t in juneTables:
    insertTable(t)

,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA2021-2-22,G8V1900010466UU4,Pass,RESET AND PASSED QC TESTING FW 9.70 6/11/21,Jordys Placido,2021-06-11,none,Mini
2,NA2021-2-22,G8V1900010466UUV,Pass,RESET AND PASSED QC TESTING FW 9.70 6/11/21,Jordys Placido,2021-06-11,none,Mini
3,NA2021-2-22,G8V19000104670W1,Pass,RESET AND PASSED QC TESTING FW 9.70 6/11/21,Jordys Placido,2021-06-11,none,Mini
4,NA2021-2-22,G8V19000104670U1,Pass,RESET AND PASSED QC TESTING FW 9.70 6/11/21,Jordys Placido,2021-06-11,none,Mini
5,NA2021-2-22,G8V19000104670VX,Pass,RESET AND PASSED QC TESTING FW 9.70 6/11/21,Jordys Placido,2021-06-11,none,Mini


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,EU/UK20210517-A15D,G8T1-9400-1132-3LM5,Pass,COMPLETED (FW: 9.70) RESET 6/14/21,Nikky Voeuth,2021-06-14,none,Mini
2,EU/UK20210517-A15D,G8T1-9400-1132-3LNT,Pass,COMPLETED (FW: 9.70) RESET 6/14/21,Nikky Voeuth,2021-06-14,none,Mini
3,EU/UK20210517-A15D,G8T1-9400-1132-3K7X,Pass,COMPLETED (FW: 9.70) RESET 6/14/21,Nikky Voeuth,2021-06-14,none,Mini
4,EU/UK20210517-A15D,G8T1-9400-1132-3KAB,Pass,COMPLETED (FW: 9.70) RESET 6/14/21,Nikky Voeuth,2021-06-14,none,Mini
5,EU/UK20210517-A15D,G8T1-9400-1132-3LQF,Pass,COMPLETED (FW: 9.70) RESET 6/14/21,Nikky Voeuth,2021-06-14,none,Mini


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA2021-05-03,G8V190001132ASRX,Pass,COMPLETED (FW 9.70) RESET 6/20/21,Jordys Placido,2021-06-20,none,Mini
2,NA2021-05-03,G8V190001132AT1U,Pass,COMPLETED (FW 9.70) RESET 6/20/21,Jordys Placido,2021-06-20,none,Mini
3,NA2021-05-03,G8V190001132ASUL,Pass,COMPLETED (FW 9.70) RESET 6/20/21,Jordys Placido,2021-06-20,none,Mini
4,NA2021-05-03,G8V190001132ARNU,Pass,COMPLETED (FW 9.70) RESET 6/20/21,Jordys Placido,2021-06-20,none,Mini
5,NA2021-05-03,G8V190001132ARPD,Pass,COMPLETED (FW 9.70) RESET 6/20/21,Jordys Placido,2021-06-20,none,Mini


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA2021-04-26,G8V19000113292SP,Pass,COMPLETE (FW 9.70) RESET 6/28/21,Jordys Placido,2021-06-28,none,Mini
2,NA2021-04-26,G8V1900011329319,Pass,COMPLETE (FW 9.70) RESET 6/28/21,Jordys Placido,2021-06-28,none,Mini
3,NA2021-04-26,G8V19000113292RH,Pass,COMPLETE (FW 9.70) RESET 6/28/21,Jordys Placido,2021-06-28,none,Mini
4,NA2021-04-26,G8V19000113292RX,Pass,COMPLETE (FW 9.70) RESET 6/28/21,Jordys Placido,2021-06-28,none,Mini
5,NA2021-04-26,G8V19000113291HH,Pass,COMPLETE (FW 9.70) RESET 6/28/21,Jordys Placido,2021-06-28,none,Mini


Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8T1-9400-1132-3JRE' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_PARSE_ERROR: 1064>, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 's of camera/camera boxes inside)//COMPLETED (FW 9.70) AND RESET 6/11/21', 'Mini'' at line 1")
success!
Error with execute: (<MySQLErrorCodes.ER_PARSE_ERROR: 1064>, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 's of camera/camera boxes inside)// COMPLETED (FW 9.70) RESET 6/15/21', 'Mini', '' at line 1")
success!
success!
success!
